In [41]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import hvplot.pandas
from sportsipy.nfl.boxscore import Boxscores, Boxscore
from sportsipy.nfl.schedule import Schedule, Game
from sportsipy.nfl.teams import Teams
pd.set_option('display.max_columns', None)

In [42]:
# Import dataframes
test_df = pd.read_csv(Path('Resources/test_df.csv'), index_col='Unnamed: 0')
train_df = pd.read_csv(Path('Resources/train_df.csv'), index_col='Unnamed: 0')
overall_df = pd.read_csv(Path('Resources/overall_df.csv'), index_col='Unnamed: 0')

In [43]:
pred_week = 13
current_week = pred_week
train_df = overall_df[overall_df['week'] < pred_week]
test_df = overall_df[overall_df['week'] == pred_week]

In [44]:
train_df_info = train_df
test_df_info = test_df

display(train_df.tail())
display(test_df.tail())

,away_name,away_abbr,home_name,home_abbr,week,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,result,elo_dif,qb_dif
158,Los Angeles Chargers,sdg,Denver Broncos,den,12,0.100000,3.800000,-0.100000,0.200000,49.800000,5.300000,2.700000,0.8,42.500000,1.800000,6.000000,-1.600000,0.300000,-5.400000,-163.900000,-1.200000,44.400000,-0.100000,16.600000,-7.300000,-0.055556,0.105924,0.0,74.721833,99.202771
159,Minnesota Vikings,min,San Francisco 49ers,sfo,12,0.000000,0.100000,0.000000,-0.400000,31.700000,7.400000,6.100000,0.6,32.000000,1.400000,0.900000,-1.100000,-0.700000,-4.000000,41.600000,-0.500000,27.700000,-0.800000,5.300000,0.300000,0.155844,0.009179,0.0,6.369632,92.115953
160,Los Angeles Rams,ram,Green Bay Packers,gnb,12,-0.027273,0.700000,-0.490909,0.445455,46.300000,3.154545,2.609091,0.4,44.918182,0.163636,4.645455,-1.336364,0.063636,-10.100000,-205.163636,-0.500000,36.200000,0.281818,-3.627273,-1.381818,-0.071795,-0.002783,0.0,-77.381374,-42.459544
161,Cleveland Browns,cle,Baltimore Ravens,rav,12,-0.154545,-3.772727,-0.027273,-0.354545,-45.963636,-6.236364,-4.272727,-0.4,-47.590909,1.472727,-2.518182,-1.690909,0.345455,5.818182,-187.427273,-0.581818,-40.145455,-0.300000,15.800000,-1.627273,-0.257576,0.020200,0.0,-51.409517,-103.082625
162,Seattle Seahawks,sea,Washington Football Team,was,12,-0.100000,-4.100000,-0.700000,-0.500000,-24.700000,-4.800000,-3.100000,0.0,-13.700000,-0.500000,-1.800000,-5.000000,0.100000,-24.500000,-336.400000,1.000000,-49.200000,-0.900000,-9.300000,11.000000,-0.355072,-0.068234,0.0,26.031709,32.264597


,away_name,away_abbr,home_name,home_abbr,week,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,result,elo_dif,qb_dif
172,Washington Football Team,was,Las Vegas Raiders,rai,13,-0.090909,1.454545,0.272727,0.090909,-72.181818,-4.000000,-2.818182,-0.090909,-71.636364,-2.454545,-2.727273,5.818182,-0.181818,36.454545,81.818182,-0.090909,-35.727273,0.363636,-20.454545,0.545455,0.058462,0.048097,1.0,15.185078,-68.820114
173,Jacksonville Jaguars,jax,Los Angeles Rams,ram,13,-0.454545,-2.636364,0.727273,0.000000,-81.181818,-0.545455,-3.181818,-1.636364,-83.727273,2.272727,-11.454545,-0.727273,0.272727,19.909091,-60.545455,0.272727,-61.272727,0.454545,13.909091,-2.545455,0.121429,-0.065264,0.0,-285.686464,-102.890676
174,Baltimore Ravens,rav,Pittsburgh Steelers,pit,13,0.227273,4.909091,-0.090909,0.545455,4.363636,-4.272727,-2.545455,0.000000,9.636364,-0.636364,3.545455,9.090909,0.454545,63.636364,258.090909,1.090909,68.000000,0.272727,5.000000,5.272727,0.384615,-0.060289,0.0,107.685364,27.419825
175,San Francisco 49ers,sfo,Seattle Seahawks,sea,13,0.272727,4.818182,0.272727,0.272727,31.000000,1.000000,0.272727,-0.090909,16.636364,0.363636,6.454545,7.909091,0.636364,38.363636,388.181818,-1.272727,69.363636,0.727273,19.727273,-14.363636,0.404762,0.057812,0.0,65.371317,-13.986310
176,New England Patriots,nwe,Buffalo Bills,buf,13,0.030303,-1.500000,-0.636364,-0.333333,-31.000000,-5.416667,-2.280303,-0.772727,-24.727273,-1.598485,-1.636364,1.924242,0.075758,-2.772727,-7.659091,0.454545,-33.939394,-0.204545,-11.689394,6.272727,0.303030,-0.046053,0.0,-26.640118,-110.185754


In [45]:
# Drop non-numerical columns
train_df = train_df.drop(columns=['away_name','away_abbr','home_name','home_abbr','week'])
test_df = test_df.drop(columns=['away_name','away_abbr','home_name','home_abbr','week'])
display(test_df.head())
display(train_df.tail())

,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,result,elo_dif,qb_dif
163,0.181818,4.636364,0.727273,0.090909,94.181818,7.181818,8.181818,0.090909,93.272727,2.363636,6.272727,-0.454545,0.090909,16.272727,86.181818,0.090909,110.454545,0.272727,25.636364,-0.909091,-0.062745,0.043014,1.0,-39.950244,159.962301
164,0.454545,4.090909,0.636364,-0.181818,86.090909,4.181818,6.727273,1.090909,80.000000,-0.090909,11.909091,2.000000,0.727273,-3.272727,166.909091,-1.000000,82.818182,-0.363636,-0.909091,-6.090909,0.335165,0.100212,1.0,171.867039,93.922882
165,0.272727,5.545455,-0.181818,-0.454545,80.454545,6.727273,5.272727,1.272727,75.818182,0.909091,13.454545,-1.181818,0.545455,10.818182,118.909091,-0.545455,91.272727,-0.272727,15.363636,-4.636364,0.188312,0.079343,1.0,237.310235,103.270532
166,0.143939,0.863636,0.250000,-1.060606,-52.015152,-9.045455,-6.068182,-0.280303,-63.257576,-0.242424,7.242424,8.825758,0.772727,71.916667,-49.515152,-1.068182,19.901515,-1.098485,-2.174242,-11.242424,-0.180952,0.049692,1.0,145.251604,103.227714
167,-0.053030,0.090909,-0.469697,-0.015152,-4.030303,-2.878788,-2.772727,-0.250000,-6.939394,-0.848485,-1.136364,-0.196970,-0.113636,10.651515,-63.128788,-0.325758,6.621212,-0.393939,-4.636364,-2.909091,-0.129412,-0.058413,0.0,-57.912337,-37.978877


,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,result,elo_dif,qb_dif
158,0.100000,3.800000,-0.100000,0.200000,49.800000,5.300000,2.700000,0.8,42.500000,1.800000,6.000000,-1.600000,0.300000,-5.400000,-163.900000,-1.200000,44.400000,-0.100000,16.600000,-7.300000,-0.055556,0.105924,0.0,74.721833,99.202771
159,0.000000,0.100000,0.000000,-0.400000,31.700000,7.400000,6.100000,0.6,32.000000,1.400000,0.900000,-1.100000,-0.700000,-4.000000,41.600000,-0.500000,27.700000,-0.800000,5.300000,0.300000,0.155844,0.009179,0.0,6.369632,92.115953
160,-0.027273,0.700000,-0.490909,0.445455,46.300000,3.154545,2.609091,0.4,44.918182,0.163636,4.645455,-1.336364,0.063636,-10.100000,-205.163636,-0.500000,36.200000,0.281818,-3.627273,-1.381818,-0.071795,-0.002783,0.0,-77.381374,-42.459544
161,-0.154545,-3.772727,-0.027273,-0.354545,-45.963636,-6.236364,-4.272727,-0.4,-47.590909,1.472727,-2.518182,-1.690909,0.345455,5.818182,-187.427273,-0.581818,-40.145455,-0.300000,15.800000,-1.627273,-0.257576,0.020200,0.0,-51.409517,-103.082625
162,-0.100000,-4.100000,-0.700000,-0.500000,-24.700000,-4.800000,-3.100000,0.0,-13.700000,-0.500000,-1.800000,-5.000000,0.100000,-24.500000,-336.400000,1.000000,-49.200000,-0.900000,-9.300000,11.000000,-0.355072,-0.068234,0.0,26.031709,32.264597


In [46]:
# Separate Features from labels.
X_train = train_df.drop(columns='result')
y_train = train_df['result']
X_test = test_df.drop(columns='result')
y_test = test_df['result']

display(X_train.head())
display(y_train.head())
display(X_test.head())
display(y_test.head())
X_train.shape

,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,elo_dif,qb_dif
0,0.0,4.0,-2.0,0.0,121.0,16.0,8.0,0.0,132.0,-4.0,-3.0,-7.0,1.0,-66.0,55.0,1.0,55.0,0.0,-9.0,11.0,-0.666667,0.200000,-50.848658,100.316973
1,1.0,-4.0,-1.0,-1.0,29.0,-13.0,-9.0,2.0,45.0,0.0,13.0,10.0,-1.0,15.0,-23.0,2.0,44.0,-2.0,-20.0,16.0,0.666667,-0.240260,-51.742731,34.616025
2,0.0,4.0,-1.0,0.0,-119.0,-14.0,-9.0,4.0,-128.0,0.0,19.0,12.0,-1.0,60.0,196.0,-1.0,-59.0,-1.0,-2.0,-9.0,1.000000,0.214286,204.627654,54.983874
3,0.0,10.0,0.0,1.0,232.0,24.0,16.0,1.0,247.0,5.0,10.0,0.0,2.0,7.0,445.0,1.0,239.0,1.0,30.0,15.0,0.000000,0.133333,-62.282436,-14.386838
4,0.0,-3.0,0.0,0.0,50.0,-9.0,-9.0,-1.0,55.0,-7.0,9.0,-3.0,1.0,-42.0,-270.0,0.0,8.0,2.0,-10.0,5.0,0.000000,-0.128205,36.874709,-13.318243


0    0.0
1    0.0
2    0.0
3    1.0
4    1.0
Name: result, dtype: float64

,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,elo_dif,qb_dif
163,0.181818,4.636364,0.727273,0.090909,94.181818,7.181818,8.181818,0.090909,93.272727,2.363636,6.272727,-0.454545,0.090909,16.272727,86.181818,0.090909,110.454545,0.272727,25.636364,-0.909091,-0.062745,0.043014,-39.950244,159.962301
164,0.454545,4.090909,0.636364,-0.181818,86.090909,4.181818,6.727273,1.090909,80.000000,-0.090909,11.909091,2.000000,0.727273,-3.272727,166.909091,-1.000000,82.818182,-0.363636,-0.909091,-6.090909,0.335165,0.100212,171.867039,93.922882
165,0.272727,5.545455,-0.181818,-0.454545,80.454545,6.727273,5.272727,1.272727,75.818182,0.909091,13.454545,-1.181818,0.545455,10.818182,118.909091,-0.545455,91.272727,-0.272727,15.363636,-4.636364,0.188312,0.079343,237.310235,103.270532
166,0.143939,0.863636,0.250000,-1.060606,-52.015152,-9.045455,-6.068182,-0.280303,-63.257576,-0.242424,7.242424,8.825758,0.772727,71.916667,-49.515152,-1.068182,19.901515,-1.098485,-2.174242,-11.242424,-0.180952,0.049692,145.251604,103.227714
167,-0.053030,0.090909,-0.469697,-0.015152,-4.030303,-2.878788,-2.772727,-0.250000,-6.939394,-0.848485,-1.136364,-0.196970,-0.113636,10.651515,-63.128788,-0.325758,6.621212,-0.393939,-4.636364,-2.909091,-0.129412,-0.058413,-57.912337,-37.978877


163    1.0
164    1.0
165    1.0
166    1.0
167    0.0
Name: result, dtype: float64

(163, 24)

In [47]:
# Scale the X data
# Instantiate the scaler
X_scaler = StandardScaler()

# Fit the scaler to Train data
X_scaler.fit(X_train)

#Scale the features data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [48]:
# Build the NN model.
number_input_features = len(X_train.columns)

hidden_nodes_layer1 = (number_input_features + 1)//2

hidden_nodes_layer2 = (hidden_nodes_layer1 + 1)//2

hidden_nodes_layer3 = (hidden_nodes_layer2 + 1)//2

display((f'Number of input features: {number_input_features}'))
display((f'Layer 1: {hidden_nodes_layer1}'))
display((f'Layer 2: {hidden_nodes_layer2}'))
display((f'Layer 3: {hidden_nodes_layer3}'))

'Number of input features: 24'

'Layer 1: 12'

'Layer 2: 6'

'Layer 3: 3'

In [49]:
# Create Sequential NN_model
nn = Sequential()

#Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Add the second layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the third layer
nn.add(Dense(units=hidden_nodes_layer3, activation="relu"))

# Add the output layer
nn.add(Dense(units=1, activation="sigmoid"))

In [50]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 12)                300       
                                                                 
 dense_9 (Dense)             (None, 6)                 78        
                                                                 
 dense_10 (Dense)            (None, 3)                 21        
                                                                 
 dense_11 (Dense)            (None, 1)                 4         
                                                                 
Total params: 403
Trainable params: 403
Non-trainable params: 0
_________________________________________________________________


In [51]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Fit the model
nfl_nn_model = nn.fit(X_train_scaled, y_train, epochs=500)

Epoch 1/500
6/6 [==============================] - 1s 5ms/step - loss: 0.7065 - accuracy: 0.4479
Epoch 2/500
6/6 [==============================] - 0s 3ms/step - loss: 0.6966 - accuracy: 0.4785
Epoch 3/500
6/6 [==============================] - 0s 3ms/step - loss: 0.6914 - accuracy: 0.5215
Epoch 4/500
6/6 [==============================] - 0s 4ms/step - loss: 0.6866 - accuracy: 0.5399
Epoch 5/500
6/6 [==============================] - 0s 3ms/step - loss: 0.6840 - accuracy: 0.5521
Epoch 6/500
6/6 [==============================] - 0s 4ms/step - loss: 0.6821 - accuracy: 0.5644
Epoch 7/500
6/6 [==============================] - 0s 3ms/step - loss: 0.6806 - accuracy: 0.5890
Epoch 8/500
6/6 [==============================] - 0s 3ms/step - loss: 0.6791 - accuracy: 0.5951
Epoch 9/500
6/6 [==============================] - 0s 3ms/step - loss: 0.6780 - accuracy: 0.6074
Epoch 10/500
6/6 [==============================] - 0s 4ms/step - loss: 0.6768 - accuracy: 0.6196
Epoch 11/500
6/6 [===========

In [52]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - loss: 1.2221 - accuracy: 0.6429 - 278ms/epoch - 278ms/step
Loss: 1.2221219539642334, Accuracy: 0.6428571343421936


In [53]:
y_pred = nn.predict(X_test_scaled)

y_pred

array([[0.96013045],
       [0.89552605],
       [0.8069068 ],
       [0.2356225 ],
       [0.04881653],
       [0.00655988],
       [0.82601637],
       [0.31679475],
       [0.15905163],
       [0.55202955],
       [0.06406483],
       [0.92709106],
       [0.9233395 ],
       [0.9990735 ]], dtype=float32)

In [54]:
def get_schedule(year):
    weeks = list(range(1,19))
    schedule_df = pd.DataFrame()
    for w in range(len(weeks)):
        date_string = str(weeks[w]) + '-' + str(year)
        week_scores = Boxscores(weeks[w],year)
        week_games_df = pd.DataFrame()
        for g in range(len(week_scores.games[date_string])):
            game = pd.DataFrame(week_scores.games[date_string][g], index = [0])[['away_name', 'away_abbr','home_name', 'home_abbr','winning_name', 'winning_abbr' ]]
            game['week'] = weeks[w]
            week_games_df = pd.concat([week_games_df,game])
        schedule_df = pd.concat([schedule_df, week_games_df]).reset_index().drop(columns = 'index') 
    return schedule_df 

In [55]:
test_df.head()

,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,result,elo_dif,qb_dif
163,0.181818,4.636364,0.727273,0.090909,94.181818,7.181818,8.181818,0.090909,93.272727,2.363636,6.272727,-0.454545,0.090909,16.272727,86.181818,0.090909,110.454545,0.272727,25.636364,-0.909091,-0.062745,0.043014,1.0,-39.950244,159.962301
164,0.454545,4.090909,0.636364,-0.181818,86.090909,4.181818,6.727273,1.090909,80.000000,-0.090909,11.909091,2.000000,0.727273,-3.272727,166.909091,-1.000000,82.818182,-0.363636,-0.909091,-6.090909,0.335165,0.100212,1.0,171.867039,93.922882
165,0.272727,5.545455,-0.181818,-0.454545,80.454545,6.727273,5.272727,1.272727,75.818182,0.909091,13.454545,-1.181818,0.545455,10.818182,118.909091,-0.545455,91.272727,-0.272727,15.363636,-4.636364,0.188312,0.079343,1.0,237.310235,103.270532
166,0.143939,0.863636,0.250000,-1.060606,-52.015152,-9.045455,-6.068182,-0.280303,-63.257576,-0.242424,7.242424,8.825758,0.772727,71.916667,-49.515152,-1.068182,19.901515,-1.098485,-2.174242,-11.242424,-0.180952,0.049692,1.0,145.251604,103.227714
167,-0.053030,0.090909,-0.469697,-0.015152,-4.030303,-2.878788,-2.772727,-0.250000,-6.939394,-0.848485,-1.136364,-0.196970,-0.113636,10.651515,-63.128788,-0.325758,6.621212,-0.393939,-4.636364,-2.909091,-0.129412,-0.058413,0.0,-57.912337,-37.978877


In [56]:
# Create dataframe to hold prediction results
results_df = test_df_info[['week','away_name','home_name']]
results_df['away_team_win_%'] = y_pred
results_df['home_team_win_%'] = 1 - results_df['away_team_win_%']
results_df['predicted_winner'] = 0

# Loop through results dataframe and add predicted_winner name to column
for index, row in results_df.iterrows():
    if results_df.loc[index,'away_team_win_%'] > results_df.loc[index,'home_team_win_%']:
        results_df.loc[index,'predicted_winner'] = results_df.loc[index,'away_name']
    else:
        results_df.loc[index,'predicted_winner'] = results_df.loc[index,'home_name']

# Set Index to the 'week' column
results_df=results_df.set_index('week')      

# Use get_schedule function to pull dataframe that shows actual weekly winners. Includes all weeks.
weekly_winner = get_schedule('2021')
weekly_winner = weekly_winner.drop(columns=['away_abbr','home_abbr','winning_abbr'])
weekly_winner = weekly_winner.rename(columns={'away_name':'ww_away_name','home_name':'ww_home_name'})

# Slices weekly_winner dataframe based on current week model is predicting.
current_week_wins = weekly_winner.loc[weekly_winner['week'] == current_week]
current_week_wins = current_week_wins.set_index('week')

# Sort both dataframes by away_name
results_df = results_df.sort_values(by='away_name')
current_week_wins = current_week_wins.sort_values(by='ww_away_name')

# Combine results & weekly_winner df's. Drop dupe columns and rename winning_name to 'actual_winner'
actual_results_df = (pd.concat([results_df,current_week_wins],axis=1)).drop(columns=['ww_away_name','ww_home_name']).rename(columns={'winning_name':'actual_winner'})
actual_results_df


C:\Users\14065\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\14065\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\14065\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

,away_name,home_name,away_team_win_%,home_team_win_%,predicted_winner,actual_winner
week,,,,,,
13,Arizona Cardinals,Chicago Bears,0.895526,0.104474,Arizona Cardinals,Arizona Cardinals
13,Baltimore Ravens,Pittsburgh Steelers,0.927091,0.072909,Baltimore Ravens,Pittsburgh Steelers
13,Dallas Cowboys,New Orleans Saints,0.960130,0.039870,Dallas Cowboys,Dallas Cowboys
13,Denver Broncos,Kansas City Chiefs,0.006560,0.993440,Kansas City Chiefs,Kansas City Chiefs
13,Indianapolis Colts,Houston Texans,0.826016,0.173984,Indianapolis Colts,Indianapolis Colts
13,Jacksonville Jaguars,Los Angeles Rams,0.064065,0.935935,Los Angeles Rams,Los Angeles Rams
13,Los Angeles Chargers,Cincinnati Bengals,0.159052,0.840948,Cincinnati Bengals,Los Angeles Chargers
13,Minnesota Vikings,Detroit Lions,0.316795,0.683205,Detroit Lions,Detroit Lions
13,New England Patriots,Buffalo Bills,0.999074,0.000926,New England Patriots,New England Patriots


In [57]:
actual_results_df['Accuracy'] = np.where(actual_results_df['predicted_winner'] == actual_results_df['actual_winner'], 1, 0)

actual_results_df = actual_results_df.dropna()

actual_results_df

,away_name,home_name,away_team_win_%,home_team_win_%,predicted_winner,actual_winner,Accuracy
week,,,,,,,
13,Arizona Cardinals,Chicago Bears,0.895526,0.104474,Arizona Cardinals,Arizona Cardinals,1
13,Baltimore Ravens,Pittsburgh Steelers,0.927091,0.072909,Baltimore Ravens,Pittsburgh Steelers,0
13,Dallas Cowboys,New Orleans Saints,0.960130,0.039870,Dallas Cowboys,Dallas Cowboys,1
13,Denver Broncos,Kansas City Chiefs,0.006560,0.993440,Kansas City Chiefs,Kansas City Chiefs,1
13,Indianapolis Colts,Houston Texans,0.826016,0.173984,Indianapolis Colts,Indianapolis Colts,1
13,Jacksonville Jaguars,Los Angeles Rams,0.064065,0.935935,Los Angeles Rams,Los Angeles Rams,1
13,Los Angeles Chargers,Cincinnati Bengals,0.159052,0.840948,Cincinnati Bengals,Los Angeles Chargers,0
13,Minnesota Vikings,Detroit Lions,0.316795,0.683205,Detroit Lions,Detroit Lions,1
13,New England Patriots,Buffalo Bills,0.999074,0.000926,New England Patriots,New England Patriots,1


In [58]:
#Week Accuracy 
accuracy = sum(actual_results_df['Accuracy'])/len(actual_results_df)
round(accuracy,2)

0.71

In [59]:
# Export actual_results_df to csv based on the current_week predicted.
file_path = str(f'Resources/week_{current_week}_base_results.csv')
actual_results_df.to_csv(Path(file_path))